In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
response = requests.get("https://www.larvalabs.com/cryptopunks/leaderboard")
soup = BeautifulSoup(response.text, "html.parser")
# print(soup.text)
# print(soup.prettify)

In [3]:
crypto_owners = soup.find(class_="table")
# crypto_owners.text

In [4]:
table_rows = crypto_owners.find_all("tr")

lines = []
items =[]
for row in table_rows: 
    attributes = row.find_all(["th","td"])
    items = []
    for att in attributes:
        items.append(att.text.strip())
    lines.append(items)
# print (lines)

In [5]:
punk_owners_df = pd.DataFrame(lines[1:], columns=lines[0])
punk_owners_df.to_csv('./CSV/owners.csv')
punk_owners_df

,#,Account,OpenSea / ENS,Number Owned,last Active
0,1,0xc352b534e8b98,,429,12 days ago
1,2,0xb7f7f6c52f2e2,WrappedCryptoPu,310,17 hours ago
2,3,0xa25803ab86a32,wilcox.eth,254,1 month ago
3,4,0xb88f61e6fbda8,,215,4 months ago
4,5,0x577ebc5de943e,,173,26 days ago
...,...,...,...,...,...
3107,3108,0x9fdf724bcdf93,cuy.eth,1,8 months ago
3108,3109,0x4dc8a4facf0cb,ilovemattyice.e,1,1 month ago
3109,3110,0x52782c2ab1741,,1,25 days ago
3110,3111,0xb76e77568756e,,1,21 days ago


In [6]:
import time
from selenium.webdriver import Firefox
from selenium.webdriver import ActionChains
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.command import Command
from selenium.common.exceptions import TimeoutException

In [ ]:
stop_id = 0
transactions_historic = []

In [38]:
browser = Firefox()
for i in range(stop_id,10000): # Al reiniciar el loop continua desde donde se ha parado
    browser.get(f'https://defypunk.com/punks/{i+1}')
        
    try:
        sales_table = WebDriverWait(browser,20).until(EC.presence_of_element_located((By.XPATH,"/html/body/div/div/main/div/div/div/div[2]/div[2]/div/table/tbody")))
        rows = sales_table.find_elements_by_tag_name('tr')
        punk_id = browser.find_element_by_tag_name('h1').text 
        stop_id = int(punk_id[12:])
        trans_by_punk = []

        for row in rows:
            attributes = row.find_elements_by_tag_name('td')
            unique_trans = [punk_id]
            for i in range(len(attributes)):
                unique_trans.append(attributes[i].text)
            transactions_historic.append(unique_trans)

    except TimeoutException:
        print("TIMEOUT!!!")
        browser.quit()

browser.quit()
print(stop_id)

0


In [40]:
trans_df = pd.DataFrame.from_records(transactions_historic)
trans_df.rename(columns={'0':'Punk_id', '1':'Trans_Type','2':'From','3':'To','4':'Amount','5':'Date'})
trans_df

,0,1,2,3,4,5
0,CryptoPunk #1,Bid withdrawn,0xE1E492,,10.0Ξ,"Aug 13, 2021 0xdd0b90"
1,CryptoPunk #1,Bid,0xE1E492,,10.0Ξ,"Aug 12, 2021 0x919480"
2,CryptoPunk #1,Transfer,0xcf6165,0xb88f61,,"Nov 30, 2020 0x9ac3a7"
3,CryptoPunk #1,Sold,0xeE3766,0xcf6165,60.0Ξ,"Nov 30, 2020 0xf4af55"
4,CryptoPunk #1,Bid,0xcf6165,,60.0Ξ,"Nov 30, 2020 0x7399ca"
...,...,...,...,...,...,...
166427,CryptoPunk #0,Bid,0x717403,,0.050000000000000007Ξ,"Jun 25, 2017 0xdfbce0"
166428,CryptoPunk #0,Sold,0xC352B5,0x00d7c9,0.98Ξ,"Jun 24, 2017 0x24893e"
166429,CryptoPunk #0,Bid,0x00d7C9,,0.98Ξ,"Jun 24, 2017 0xc29572"
166430,CryptoPunk #0,Bid,0x948a10,,0.67Ξ,"Jun 24, 2017 0xbe1685"


In [41]:
stop_id

0

In [42]:
trans_df.to_csv('./CSV/punks_transactions.csv')